# Expected Loss Calculation

In [1]:
%load_ext autoreload
%autoreload 2

## Change to project directory (google colab)

In [2]:
import os
os.chdir('/content/drive/MyDrive/projects/credit-risk-modelling-udemy/notebooks')
os.listdir('../')

['notebooks',
 'venv-hpx360-win',
 'data',
 'src',
 '.git',
 'requirements.txt',
 'README.md',
 'src.egg-info',
 'setup.py',
 'reports',
 'outputs',
 '.gitignore']

In [5]:
!pip install -e ../.

Obtaining file:///content/drive/MyDrive/projects/credit-risk-modelling-udemy
  Running setup.py develop for src


## Some imports

In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import src
from src.project.config import PROJECT_DIR

## Load Data

### For LGD and EAD Models

In [4]:
fpath = os.path.join(PROJECT_DIR, 'outputs', 'loan_data_general_preprocessed_nb2.pkl')
loan_data_prep = pd.read_pickle(fpath)
loan_data_prep.head()

,index,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,...,addr_state:IA,addr_state:KS,addr_state:KY,addr_state:LA,addr_state:ME,addr_state:MD,addr_state:MA,addr_state:MI,addr_state:MN,addr_state:MS,addr_state:MO,addr_state:MT,addr_state:NE,addr_state:NV,addr_state:NH,addr_state:NJ,addr_state:NM,addr_state:NY,addr_state:NC,addr_state:ND,addr_state:OH,addr_state:OK,addr_state:OR,addr_state:PA,addr_state:RI,addr_state:SC,addr_state:SD,addr_state:TN,addr_state:TX,addr_state:UT,addr_state:VT,addr_state:VA,addr_state:WA,addr_state:WV,addr_state:WI,addr_state:WY,addr_state:DC,initial_list_status:f,initial_list_status:w,good_bad_loan
0,0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,2011-12-01,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I need to upgrade...,credit_card,Computer,860xx,AZ,27.65,0.0,1985-01-01,1.0,NaN,NaN,3.0,0.0,13648,83.7,9.0,f,0.0,0.0,5861.071414,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
1,1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,2011-12-01,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I plan to use thi...,car,bike,309xx,GA,1.00,0.0,1999-04-01,5.0,NaN,NaN,3.0,0.0,1687,9.4,4.0,f,0.0,0.0,1008.710000,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,2011-12-01,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,NaN,small_business,real estate business,606xx,IL,8.72,0.0,2001-11-01,2.0,NaN,NaN,2.0,0.0,2956,98.5,10.0,f,0.0,0.0,3003.653644,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
3,3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,2011-12-01,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/21/11 > to pay for proper...,other,personel,917xx,CA,20.00,0.0,1996-02-01,1.0,35.0,NaN,10.0,0.0,5598,21.0,37.0,f,0.0,0.0,12226.302210,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
4,4,1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,2011-12-01,Current,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/21/11 > I plan on combini...,other,Personal,972xx,OR,17.94,0.0,1996-01-01,0.0,38.0,NaN,15.0,0.0,27783,53.9,38.0,f,766.9,766.9,3242.170000,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1


In [5]:
loan_data_prep['mths_since_last_delinq'].fillna(0, inplace = True)
loan_data_prep['mths_since_last_record'].fillna(0, inplace = True)


### For PD Model

In [39]:
fpaths = {subset: os.path.join(PROJECT_DIR, 'outputs', f'loan_data_2007_2014_{subset}_prepared_for_pd_model_nb3.pkl') for subset in ['train', 'test']}

In [36]:
loan_data_inputs_train_pd = pd.read_pickle(fpaths['train'])
loan_data_inputs_test_pd = pd.read_pickle(fpaths['test'])
loan_data_inputs_pd = pd.concat([loan_data_inputs_train_pd, loan_data_inputs_test_pd]).reset_index(drop=True)
loan_data_inputs_pd.head()

,index,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,...,int_rate:<=9.548,int_rate:9.548-12.025,int_rate:12.025-15.327,int_rate:15.327-19.868,int_rate:>=19.868,funded_amnt_binned,annual_inc_binned,annual_inc_combined,annual_inc:<=28000,annual_inc:28000-45000,annual_inc:45000-49000,annual_inc:49000-62983,annual_inc:62983-67000,annual_inc:67000-81000,annual_inc:81000-105000,annual_inc:>=105000,mths_since_last_delinq_binned,mths_since_last_delinq_combined,mths_since_last_delinq:missing,mths_since_last_delinq:0-4,mths_since_last_delinq:4-31,mths_since_last_delinq:31-57,mths_since_last_delinq:>=57,dti_binned,dti_combined,dti:<=7.598,dti:7.598-11.997,dti:11.997-16.796,dti:16.796-21.595,dti:21.595-25.994,dti:>=25.994,mths_since_last_record_binned,mths_since_last_record_combined,mths_since_last_record:missing,mths_since_last_record:0-2,mths_since_last_record:2-21,mths_since_last_record:21-32,mths_since_last_record:32-81,mths_since_last_record:81-87,mths_since_last_record:>=87
0,427211,12796369,14818505,24000,24000,24000.0,36 months,8.90,762.08,A,A5,Supervisor inventory management,3 years,MORTGAGE,77000.0,Source Verified,2014-03-01,Current,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 03/12/14 > I have 5 credit c...,debt_consolidation,Debt consolidation,295xx,SC,21.91,0.0,1986-12-01,1.0,NaN,NaN,20.0,0.0,30489,53.5,32.0,f,10098.30,10098.30,16765.76000,...,1,0,0,0,0,"(23960.0, 24650.0]","(75000.0, 81000.0]",67000-81000,0,0,0,0,0,1,0,0,NaN,missing,1,0,0,0,0,"(21.595, 21.995]",21.595-25.994,0,0,0,0,1,0,NaN,missing,1,0,0,0,0,0,0
1,206088,1439740,1691948,10000,10000,10000.0,36 months,14.33,343.39,C,C1,mizuho corporate bank,6 years,MORTGAGE,112000.0,Not Verified,2012-08-01,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 07/23/12 > I was looking for...,debt_consolidation,Credit card consolidation,070xx,NJ,7.49,1.0,1997-12-01,2.0,18.0,NaN,15.0,0.0,15836,53.1,38.0,f,0.00,0.00,12357.02066,...,0,0,1,0,0,"(9470.0, 10160.0]","(105000.0, 120000.0]",>=105000,0,0,0,0,0,0,0,1,"(15.04, 18.8]",4-31,0,0,1,0,0,"(7.198, 7.598]",<=7.598,1,0,0,0,0,0,NaN,missing,1,0,0,0,0,0,0
2,136020,5214749,6556909,20425,20425,20425.0,36 months,8.90,648.56,A,A5,Internal Medicine of Griffin,10+ years,MORTGAGE,84000.0,Verified,2013-06-01,Current,n,https://www.lendingclub.com/browse/loanDetail....,NaN,debt_consolidation,Lend Club,302xx,GA,14.83,0.0,1991-07-01,1.0,46.0,NaN,9.0,0.0,29813,89.5,20.0,f,3183.62,3183.62,20090.40000,...,1,0,0,0,0,"(19820.0, 20510.0]","(81000.0, 88800.0]",81000-105000,0,0,0,0,0,0,1,0,"(45.12, 48.88]",31-57,0,0,0,1,0,"(14.796, 15.196]",11.997-16.796,0,0,1,0,0,0,NaN,missing,1,0,0,0,0,0,0
3,412305,13827698,15890016,17200,17200,17200.0,36 months,16.59,609.73,D,D1,Administrative Assistant,7 years,RENT,43000.0,Source Verified,2014-04-01,Late (31-120 days),n,https://www.lendingclub.com/browse/loanDetail....,NaN,debt_consolidation,Debt consolidation,015xx,MA,13.68,1.0,1998-10-01,3.0,6.0,NaN,9.0,0.0,7523,60.2,13.0,f,9459.50,9459.50,11615.36000,...,0,0,0,1,0,"(17060.0, 17750.0]","(41600.0, 45000.0]",28000-45000,0,1,0,0,0,0,0,0,"(3.76, 7.52]",4-31,0,0,1,0,0,"(13.597, 13.997]",11.997-16.796,0,0,1,0,0,0,NaN,missing,1,0,0,0,0,0,0
4,36159,422455,496525,8400,8400,7450.0,36 months,12.84,282.40,C,C2,Bank of A,5 years,MORTGAGE,94000.0,Verified,2009-07-01,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,Unexpectd California tuition hike - Need help ...,educational,Student Loan,913xx,CA,22.54,0.0,1998-07-01,1.0,NaN,NaN,14.0,0.0,65621,81.5,30.0,f,0.00,0.00,5422.21000,...,0,0,1,0,0,"(8090.0, 8780.0]","(88800.0, 95900.0]",81000-

In [41]:
import json

infile = os.path.join(PROJECT_DIR, 'outputs', 'dummy_vars_ref_cats.json')
with open(infile, 'r') as f:
    d = json.load(f)

DUMMY_VARIABLES_PD = d['DUMMY_VARIABLES']
REFERENCE_CATEGORIES_PD = d['REFERENCE_CATEGORIES']

## Load Models

In [19]:
from src.models import LogisticRegressionWithPValues, LinearRegressionWithPValues
import pickle

### PD Model

In [21]:
fpath = os.path.join(PROJECT_DIR, 'outputs', 'pd_model_logreg.pkl')
with open(fpath, 'rb') as f:
    pd_model = pickle.load(f)


### LGD Model - Stage 1

In [23]:
fpath = os.path.join(PROJECT_DIR, 'outputs', 'lgd_model_stage_1.pkl')
with open(fpath, 'rb') as f:
    reg_lgd_stage_1 = pickle.load(f)


### LGD Model - Stage 2

In [22]:
fpath = os.path.join(PROJECT_DIR, 'outputs', 'lgd_model_stage_2.pkl')
with open(fpath, 'rb') as f:
    reg_lgd_stage_2 = pickle.load(f)


### EAD Model

In [30]:
fpath = os.path.join(PROJECT_DIR, 'outputs', 'ead_model.pkl')
with open(fpath, 'rb') as f:
    reg_ead = pickle.load(f)


## Prepare Inputs for `LGD` an `EAD` models

## Variable List

In [6]:
variable_lst = ['grade:A',
'grade:B',
'grade:C',
'grade:D',
'grade:E',
'grade:F',
'grade:G',
'home_ownership:MORTGAGE',
'home_ownership:NONE',
'home_ownership:OTHER',
'home_ownership:OWN',
'home_ownership:RENT',
'verification_status:Not Verified',
'verification_status:Source Verified',
'verification_status:Verified',
'purpose:car',
'purpose:credit_card',
'purpose:debt_consolidation',
'purpose:educational',
'purpose:home_improvement',
'purpose:house',
'purpose:major_purchase',
'purpose:medical',
'purpose:moving',
'purpose:other',
'purpose:renewable_energy',
'purpose:small_business',
'purpose:vacation',
'purpose:wedding',
'initial_list_status:f',
'initial_list_status:w',
'term_int',
'emp_length_int',
'mths_since_issue_d',
'mths_since_earliest_cr_line',
'funded_amnt',
'int_rate',
'installment',
'annual_inc',
'dti',
'delinq_2yrs',
'inq_last_6mths',
'mths_since_last_delinq',
'mths_since_last_record',
'open_acc',
'pub_rec',
'total_acc',
'acc_now_delinq',
'total_rev_hi_lim']


variable_ref_cat =  ['grade:G',
'home_ownership:RENT',
'verification_status:Verified',
'purpose:credit_card',
'initial_list_status:f']

In [7]:
loan_data_prep_lgd_ead = loan_data_prep[variable_lst].drop(variable_ref_cat, axis=1)

### Recovery Rate


In [26]:
loan_data_prep['recovery_rate_stage_1'] = reg_lgd_stage_1.predict(loan_data_prep_lgd_ead)  # Stage 1: predict whether recovery_rate 0 or not
loan_data_prep['recovery_rate_stage_2'] = reg_lgd_stage_2.predict(loan_data_prep_lgd_ead) # Stage 2: predict the actual recovery rate given non-zero
loan_data_prep['recovery_rate'] = loan_data_prep['recovery_rate_stage_1'] * loan_data_prep['recovery_rate_stage_2'] # Combining Stages

In [27]:
loan_data_prep['recovery_rate'] = np.where(loan_data_prep['recovery_rate'] < 0, 0, loan_data_prep['recovery_rate'])
loan_data_prep['recovery_rate'] = np.where(loan_data_prep['recovery_rate'] > 1, 1, loan_data_prep['recovery_rate'])

### LGD

In [28]:
loan_data_prep['lgd'] = 1 - loan_data_prep['recovery_rate']

### CCF

In [32]:
loan_data_prep['ccf'] = reg_ead.predict(loan_data_prep_lgd_ead)

In [34]:
loan_data_prep['ccf'] = np.where(loan_data_prep['ccf'] < 0, 0, loan_data_prep['ccf'])
loan_data_prep['ccf'] = np.where(loan_data_prep['ccf'] > 1, 1, loan_data_prep['ccf'])

### EAD

In [35]:
loan_data_prep['ead'] = loan_data_prep['ccf']*loan_data_prep['funded_amnt']

### PD

In [45]:
loan_data_inputs_pd['pd'] = pd_model.predict_proba(loan_data_inputs_pd[DUMMY_VARIABLES_PD])[:, 0]

## Combining the features of the two dataframes

In [60]:
loan_data_prep_comb = pd.concat([loan_data_prep.set_index('index'), loan_data_inputs_pd.set_index('index')], axis=1)

In [61]:
loan_data_prep_comb.shape

(466285, 462)

### EL

In [62]:
loan_data_prep_comb['el'] = loan_data_prep_comb['pd']*loan_data_prep_comb['lgd']*loan_data_prep_comb['ead']

In [70]:
loan_data_prep_comb[['funded_amnt', 'pd', 'lgd', 'ead', 'el']].head().T.drop_duplicates().T

,funded_amnt,pd,lgd,ead,el
index,,,,,
0,5000.0,0.158803,0.913771,2949.577234,428.011761
1,2500.0,0.208918,0.915505,1944.451950,371.906246
2,2400.0,0.286181,0.919479,1579.922915,415.737084
3,10000.0,0.184007,0.904920,6606.540166,1100.065263
4,3000.0,0.146707,0.911458,2124.632805,284.100569


In [72]:
loan_data_prep_comb['el'].sum()

503600595.7958808

In [71]:
loan_data_prep_comb['el'].sum() / loan_data_prep_comb['funded_amnt'].sum().drop_duplicates()


funded_amnt    0.07557
dtype: float64

* 7.67% is the Expected Loss on the loan portfolio which is a bit less than what a bank holds in its capital, which is around 10%. 
* Observed Expected Loss values are anywhere between 2% and 10%
* Depending upon the exposure, the bank management can decide to be conservative or aggressive when giving out loans